### Import libraries

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Imported Powerpoints/car-safety'    #project folder directory


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd 'gdrive/My Drive/Imported Powerpoints/car-safety'

/content/gdrive/My Drive/Imported Powerpoints/car-safety


In [0]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import json
import math
import requests
import numpy as np

pd.options.display.max_columns = None

### Getting data

#### Reading Car rating from API

In [0]:
def get_vehicle_id(year, make, model):
    response = requests.get('https://webapi.nhtsa.gov/api/SafetyRatings/modelyear/%s/make/%s/model/%s?format=json' % (year, make, model))
    if response.status_code == 200:
        json_response = response.json()
        print(json_response['Message'])
        if int(json_response['Count']) > 0:
            return json_response['Results'][0]['VehicleId']
    else:
        print ('API request failed.', response.status_code)
    return '-1'

In [36]:
vehicle_id = get_vehicle_id('2019', 'audi', 'a3')
cur_car_rating = pd.read_csv('https://webapi.nhtsa.gov/api/SafetyRatings/VehicleId/%s?format=csv' % vehicle_id)
cur_car_rating.head()

Results returned successfully


,VEHICLE_PICTURE,OVERALL_STARS,OVERALL_FRNT_STARS,FRNT_DRIV_STARS,FRNT_PASS_STARS,FRNT_FOOT_NOTES_DRIV,FRNT_FOOT_NOTES_PASS,FRNT_SAFETY_CONCERN_DRIV,FRNT_SAFETY_CONCERN_PASS,FRNT_CRASH_PICTURE,FRNT_CRASH_VIDEO,OVERALL_SIDE_STARS,SIDE_DRIV_STARS,SIDE_PASS_STARS,SIDE_FOOT_NOTES_DRIV,SIDE_FOOT_NOTES_PASS,SIDE_SAFETY_CONCERN_DRIV,SIDE_SAFETY_CONCERN_PASS,SIDE_CRASH_PICTURE,SIDE_CRASH_VIDEO,ROLLOVER_STARS,ROLLOVER_STARS_2,ROLLOVER_POSSIBILITY,ROLLOVER_POSSIBILITY_2,ROLL_FOOT_NOTES,ROLL_SAFETY_CONCERN,SIDE_POLE_STARS,POLE_FOOT_NOTES,POLE_SAFETY_CONCERN,SIDE_POLE_PICTURE,SIDE_POLE_VIDEO,NHTSA_ESC,NHTSA_FRNT_COLLISION_WARNING,NHTSA_LANE_DEPARTURE_WARNING,NHTSA_BACKUP_CAMERA,COMPLAINTS_COUNT,RECALLS_COUNT,INVESTIGATIONS_COUNT,MODEL_YR,MAKE,MODEL,MMY,VEHICLE_ID
0,http://www.safercar.gov/staticfiles/DOT/saferc...,5,4,4,5,NaN,NaN,NaN,NaN,http://www.safercar.gov/staticfiles/DOT/saferc...,http://www.safercar.gov/staticfiles/DOT/saferc...,5,5,5,NaN,These ratings do not apply to vehicles with op...,NaN,NaN,http://www.safercar.gov/staticfiles/DOT/saferc...,http://www.safercar.gov/staticfiles/DOT/saferc...,4,Not Rated,0.109,0,NaN,NaN,5,NaN,NaN,http://www.safercar.gov/staticfiles/DOT/saferc...,http://www.safercar.gov/staticfiles/DOT/saferc...,NaN,No,Optional,NaN,NaN,NaN,NaN,2019,AUDI,A3,2019 Audi A3 4 DR AWD,13688


#### Reading CSV files

In [38]:
df_tests = pd.read_csv('data/test.psv', sep = '|', encoding = 'unicode_escape')
df_vehicles = pd.read_csv('data/veh.psv', sep = '|', encoding = 'unicode_escape')
df_uk_accidents = pd.read_csv('data/uk_accidents.csv', encoding = 'unicode_escape')
df_uk_vehicles = pd.read_csv('data/uk_vehicles.csv', encoding = 'unicode_escape')
df_uk_casualties = pd.read_csv('data/uk_casualties.csv', encoding = 'unicode_escape')
df_uk_makes = pd.read_csv('data/Vehicle_Information.csv', usecols = [0, 10, 11, 19], encoding = 'unicode_escape')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocessing

In [0]:
# Appending makes and models to the UK vehicles DF
if 'Vehicle_Make' not in df_uk_vehicles:
    df_uk_vehicles = pd.merge(df_uk_vehicles, df_uk_makes, on = ['Accident_Index', 'Vehicle_Reference'], how='left')
    
# Rename vehicle make, model to follow the same convention
df_uk_vehicles.rename(columns={'make': 'Vehicle_Make', 'model': 'Vehicle_Model'}, inplace = True)


# Mapping numeric values to corresponding text
df_uk_vehicles_map = {
    'Sex_of_Driver': {-1: np.nan, 3: np.nan, 2: 'Female', 1: 'Male'},
    'Age_Band_of_Driver': {-1: np.nan, 1: '0-5', 2: '6-10', 3: '11-15', 4: '16-20', 5: '21-25', 6: '26-35', 7: '36-45', 8: '46-55', 9: '56-65', 10: '66-75', 11: 'Over 75'}, 
    'Vehicle_Make': {'MISSING': np.nan},
    'Vehicle_Model': {'MISSING': np.nan},
    'Age_of_Vehicle': {-1: np.nan},
    '1st_Point_of_Impact': {-1: np.nan, 1: 'Front', 2: 'Back', 3: 'Offside', 4: 'Nearside'},
    'Vehicle_Manoeuvre': {-1: np.nan, 1: 'Reversing', 2: 'Parked', 3: 'Waiting to go - held up', 4: 'Slowing or stopping', 5: 'Moving off', 6: 'U-turn', 7: 'Turning left', 8: 'Waiting to turn left', 9: 'Turning right', 10: 'Waiting to turn right', 11: 'Changing lane to left', 12: 'Changing lane to right', 13: 'Overtaking moving vehicle – offside', 14: 'Overtaking static vehicle - offside', 15: 'Overtaking - nearside', 16: 'Going ahead left-hand bend', 17: 'Going ahead right-hand bend', 18: 'Going ahead other'},
    'Vehicle_Type': {-1: np.nan, 90: np.nan, 1: 'Pedal cycle', 2: 'Motorcycle <=50cc', 3: 'Motorcycle <=125cc', 4: 'Motorcycle 125cc-500cc', 5: 'Motorcycle >500cc', 8: 'Taxi/Private hire car', 9: 'Car', 10: 'Minibus (8-16 seats)', 11: 'Bus (>=17 seats)', 16: 'Ridden horse', 17: 'Agricultural vehicle', 18: 'Tram', 19: 'Van <=3.5 t.', 20: 'Goods 3.5t.-7.5t', 21: 'Goods >=7.5 t.', 22: 'Mobility scooter', 23: 'Electric motorcycle', 97: 'Motorcycle - unknown cc', 98: 'Goods vehicle - unknown weight'},
    'Vehicle_Location-Restricted_Lane': {-1: np.nan, 0: 'On main carriageway - not in restricted lane', 1: 'Tram/Light rail track', 2: 'Bus lane', 3:	'Busway (including guided busway)', 4: 'Cycle lane (on main carriageway)', 5: 'Cycleway or shared use footway (not part of  main carriageway)', 6: 'On lay-by or hard shoulder', 7:	'Entering lay-by or hard shoulder', 8:	'Leaving lay-by or hard shoulder', 9: 'Footway (pavement)', 10:	'Not on carriageway'},
    'Junction_Location': {-1: np.nan, 0: 'Not at or within 20 metres of junction', 1:	'Approaching junction or waiting/parked at junction approach', 2:	'Cleared junction or waiting/parked at junction exit', 3:	'Leaving roundabout', 4: 'Entering roundabout', 5: 'Leaving main road', 6: 'Entering main road', 7:	'Entering from slip road', 8:	'Mid Junction - on roundabout or on main road'},
    'Skidding_and_Overturning': {-1: np.nan, 0: 'None', 1: 'Skidded', 2: 'Skidded and overturned', 3:	'Jackknifed', 4: 'Jackknifed and overturned', 5: 'Overturned'},
    'Hit_Object_in_Carriageway': {-1: np.nan, 11: np.nan, 0: 'None', 1: 'Previous accident', 2: 'Road works', 4: 'Parked vehicle', 5:	'Bridge (roof)', 6:	'Bridge (side)', 7:	'Bollard or refuge', 8:	'Open door of vehicle', 9: 'Central island of roundabout', 10: 'Kerb', 12: 'Any animal (except ridden horse)'}, 
    'Vehicle_Leaving_Carriageway': {-1: np.nan, 0: 'Did not leave carriageway', 1: 'Nearside', 2:	'Nearside and rebounded', 3: 'Straight ahead at junction', 4: 'Offside on to central reservation', 5:	'Offside on to centrl res + rebounded', 6: 'Offside - crossed central reservation', 7: 'Offside', 8: 'Offside and rebounded'},
    'Hit_Object_off_Carriageway': {-1: np.nan, 10: np.nan, 0: 'None', 1: 'Road sign or traffic signal', 2: 'Lamp post', 3: 'Telegraph or electricity pole', 4: 'Tree', 5:	'Bus stop or bus shelter', 6:	'Central crash barrier', 7:	'Near/Offside crash barrier', 8: 'Submerged in water', 9: 'Entered ditch', 11: 'Wall or fence'},
    'Journey_Purpose_of_Driver' : {-1: np.nan, 5: np.nan, 6: np.nan, 15: np.nan, 1:	'Journey as part of work', 2:	'Commuting to/from work', 3: 'Taking pupil to/from school', 4: 'Pupil riding to/from school'},
    'Age_of_Driver': {-1: np.nan},
    'Engine_Capacity_(CC)': {-1: np.nan},
    'Propulsion_Code': {-1: np.nan, 1: 'Petrol', 2: 'Heavy oil', 3: 'Electric', 4:	'Steam', 5:	'Gas', 6:	'Petrol/Gas (LPG)', 7: 'Gas/Bi-fuel', 8: 'Hybrid electric', 9: 'Gas Diesel', 10: 'New fuel technology', 11:	'Fuel cells', 12:	'Electric diesel'},
    'Driver_Home_Area_Type': {-1: np.nan, 1: 'Urban area', 2:	'Small town', 3: 'Rural'}
}


df_uk_accidents_map = {
    'Accident_Severity': {-1: np.nan, 1: 'Fatal', 2: 'Serious', 3: 'Slight'},
    'Weather_Conditions': {-1: np.nan, 8: np.nan, 9: np.nan, 1: 'Fine no high winds', 2: 'Raining no high winds', 3: 'Snowing no high winds', 4: 'Fine + high winds', 5: 'Raining + high winds', 6: 'Snowing + high winds', 7: 'Fog or mist'},
    'Light_Conditions': {-1: np.nan, 1: 'Daylight', 4: 'Darkness - lights lit', 5: 'Darkness - lights unlit', 6: 'Darkness - no lighting', 7: 'Darkness - lighting unknown'},
    'Police_Force': {1:	'Metropolitan Police', 3: 'Cumbria', 4:	'Lancashire', 5: 'Merseyside', 6:	'Greater Manchester', 7: 'Cheshire', 10: 'Northumbria', 11:	'Durham', 12:	'North Yorkshire', 13: 'West Yorkshire', 14: 'South Yorkshire', 16:	'Humberside', 17:	'Cleveland', 20: 'West Midlands', 21:	'Staffordshire', 22: 'West Mercia', 23:	'Warwickshire', 30: 'Derbyshire', 31:	'Nottinghamshire', 32: 'Lincolnshire', 33:	'Leicestershire', 34:	'Northamptonshire', 35:	'Cambridgeshire', 36:	'Norfolk', 37: 'Suffolk', 40:	'Bedfordshire', 41:	'Hertfordshire', 42: 'Essex', 43:	'Thames Valley', 44: 'Hampshire', 45:	'Surrey', 46:	'Kent', 47:	'Sussex', 48:	'City of London', 50:	'Devon and Cornwall', 52:	'Avon and Somerset', 53: 'Gloucestershire', 54:	'Wiltshire', 55: 'Dorset', 60: 'North Wales', 61:	'Gwent', 62: 'South Wales', 63:	'Dyfed-Powys', 91: 'Northern', 92: 'Grampian', 93: 'Tayside', 94:	'Fife', 95:	'Lothian and Borders', 96: 'Central', 97:	'Strathclyde', 98: 'Dumfries and Galloway'},
    'Day_of_Week': {1: 'Sunday', 2: 'Monday', 3: 'Tuesday', 4: 'Wednesday', 5: 'Thursday', 6: 'Friday', 7: 'Saturday'},
    '1st_Road_Class': {6: np.nan, 1: 'Motorway', 2: 'A(M)', 3:	'A', 4:	'B', 5: 'C'},
    'Road_Type': {-1: np.nan, 9: np.nan, 1: 'Roundabout', 2: 'One way street', 3: 'Dual carriageway', 6: 'Single carriageway', 7: 'Slip road', 12: 'One way street/Slip road'},
    'Junction_Detail': {-1: np.nan, 9: np.nan, 0: 'Not at junction or within 20 metres', 1:	'Roundabout', 2: 'Mini-roundabout', 3: 'T or staggered junction', 5: 'Slip road', 6: 'Crossroads', 7: 'More than 4 arms (not roundabout)', 8:	'Private drive or entrance'},
    'Junction_Control': {-1: np.nan, 0:	'Not at junction or within 20 metres', 1:	'Authorised person', 2:	'Auto traffic signal', 3:	'Stop sign', 4:	'Give way or uncontrolled'},
    '2nd_Road_Class': {-1: np.nan, 6: np.nan, 0: 'Not at junction or within 20 metres', 1: 'Motorway', 2:	'A(M)', 3: 'A', 4: 'B',	5: 'C'},
    'Pedestrian_Crossing-Human_Control': {-1: np.nan, 0: 'None within 50 metres', 1: 'Control by school crossing patrol', 2: 'Control by other authorised person'},
    'Pedestrian_Crossing-Physical_Facilities': {-1: np.nan, 0: 'No physical crossing facilities within 50 metres', 1: 'Zebra', 4:	'Pelican, puffin, toucan or similar non-junction pedestrian light crossing', 5:	'Pedestrian phase at traffic signal junction', 7:	'Footbridge or subway', 8: 'Central refuge'},
    'Road_Surface_Conditions': {-1: np.nan, 1: 'Dry', 2: 'Wet or damp', 3: 'Snow', 4:	'Frost or ice', 5: 'Flood over 3cm. deep', 6: 'Oil or diesel', 7:	'Mud'},
    'Special_Conditions_at_Site': {-1: np.nan, 0:	'None', 1: 'Auto traffic signal - out', 2: 'Auto signal part defective', 3:	'Road sign or marking defective or obscured', 4: 'Roadworks', 5: 'Road surface defective', 6:	'Oil or diesel', 7: 'Mud'},
    'Carriageway_Hazards': {-1: np.nan, 0: 'None', 1: 'Vehicle load on road', 2: 'Other object on road', 3:	'Previous accident', 4:	'Dog on road', 5:	'Other animal on road', 6: 'Pedestrian in carriageway - not injured', 7: 'Any animal in carriageway (except ridden horse)'},
    'Urban_or_Rural_Area': {3: np.nan, 1: 'Urban', 2:	'Rural'},
    'Did_Police_Officer_Attend_Scene_of_Accident': {-1: np.nan, 1: 'Yes', 2: 'No', 3: 'No - accident was reported using a self completion  form (self rep only)'}
}

df_uk_casualties_map = {
    'Casualty_Severity': {-1: np.nan, 1: 'Fatal', 2: 'Serious', 3: 'Slight'},
    'Sex_of_Casualty': {-1: np.nan, 3: np.nan, 2: 'Female', 1: 'Male'},
    'Casualty_Class': {-1: np.nan, 1: 'Driver or rider', 2: 'Passenger', 3: 'Pedestrian'},
    'Age_of_Casualty': {-1: np.nan },
    'Age_Band_of_Casualty': {-1: np.nan, 1: '0-5', 2: '6-10', 3: '11-15', 4: '16-20', 5: '21-25', 6: '26-35', 7: '36-45', 8: '46-55', 9: '56-65', 10: '66-75', 11: 'Over 75'},
    'Pedestrian_Location': {-1: np.nan, 10: np.nan, 0: 'Not a Pedestrian', 1:	'Crossing on pedestrian crossing facility', 2: 'Crossing in zig-zag approach lines', 3: 'Crossing in zig-zag exit lines', 4: 'Crossing elsewhere within 50m. of pedestrian crossing', 5: 'In carriageway, crossing elsewhere', 6:	'On footway or verge', 7:	'On refuge, central island or central reservation', 8: 'In centre of carriageway - not on refuge, island or central reservation', 9: 'In carriageway, not crossing'},
    'Pedestrian_Movement': {-1: np.nan, 9: np.nan, 0:	'Not a Pedestrian', 1: 'Crossing from driver nearside', 2: 'Crossing from nearside - masked by parked or stationary vehicle', 3: 'Crossing from driver offside', 4: 'Crossing from offside - masked by  parked or stationary vehicle', 5: 'In carriageway, stationary - not crossing  (standing or playing)', 6: 'In carriageway, stationary - not crossing  (standing or playing) - masked by parked or stationary vehicle', 7: 'Walking along in carriageway, facing traffic', 8:	'Walking along in carriageway, back to traffic'},
    'Car_Passenger': {-1: np.nan, 0: 'Not car passenger', 1: 'Front seat passenger', 2:	'Rear seat passenger'},
    'Bus_or_Coach_Passenger': {-1: np.nan, 0: 'Not a bus or coach passenger', 1: 'Boarding', 2:	'Alighting', 3:	'Standing passenger', 4: 'Seated passenger'},
    'Pedestrian_Road_Maintenance_Worker': {-1: np.nan, 2: np.nan, 0: 'No / Not applicable', 1: 'Yes'},
    'Casualty_Type': {-1:np.nan, 0:	'Pedestrian', 1: 'Cyclist', 2: 'Motorcycle 50cc and under rider or passenger', 3: 'Motorcycle 125cc and under rider or passenger', 4:	'Motorcycle over 125cc and up to 500cc rider or  passenger', 5:	'Motorcycle over 500cc rider or passenger', 8: 'Taxi/Private hire car occupant', 9:	'Car occupant', 10:	'Minibus (8 - 16 passenger seats) occupant', 11: 'Bus or coach occupant (17 or more pass seats)', 16:	'Horse rider', 17: 'Agricultural vehicle occupant', 18:	'Tram occupant', 19: 'Van / Goods vehicle (3.5 tonnes mgw or under) occupant', 20: 'Goods vehicle (over 3.5t. and under 7.5t.) occupant', 21:	'Goods vehicle (7.5 tonnes mgw and over) occupant', 22:	'Mobility scooter rider', 23:	'Electric motorcycle rider or passenger', 90:	'Other vehicle occupant', 97:	'Motorcycle - unknown cc rider or passenger', 98: 'Goods vehicle (unknown weight) occupant'}
}

for key in df_uk_vehicles_map:
    for old_value in df_uk_vehicles_map[key]:
        df_uk_vehicles[key] = df_uk_vehicles[key].replace(old_value, df_uk_vehicles_map[key][old_value])
for key in df_uk_accidents_map:
    for old_value in df_uk_accidents_map[key]:
        df_uk_accidents[key] = df_uk_accidents[key].replace(old_value, df_uk_accidents_map[key][old_value])
for key in df_uk_casualties_map:
    for old_value in df_uk_casualties_map[key]:
        df_uk_casualties[key] = df_uk_casualties[key].replace(old_value, df_uk_casualties_map[key][old_value])

# Converting UK Accident_Index to string
df_uk_vehicles.Accident_Index = df_uk_vehicles.Accident_Index.astype(str)
df_uk_accidents.Accident_Index = df_uk_accidents.Accident_Index.astype(str)
df_uk_casualties.Accident_Index = df_uk_casualties.Accident_Index.astype(str)

# Converting UK number columns to floats
df_uk_vehicles.Age_of_Vehicle = df_uk_vehicles.Age_of_Vehicle.astype(float)

# Calculating vehicle year from age_of_vehicle and accident date
if 'Vehicle_Year' not in df_uk_vehicles:
    df_uk_vehicles = pd.merge(df_uk_vehicles, df_uk_accidents[['Accident_Index','Date']], on = ['Accident_Index'], how='left')
    df_uk_vehicles['Vehicle_Year'] = df_uk_vehicles.Date.str.extract(r'([0-9][0-9][0-9][0-9])', expand=True)
    df_uk_vehicles['Vehicle_Year'] = df_uk_vehicles['Vehicle_Year'].astype(float)
    df_uk_vehicles['Vehicle_Year'] = df_uk_vehicles['Vehicle_Year'] - df_uk_vehicles.Age_of_Vehicle  
    
# Dropping unneccessary columns
df_uk_vehicles.drop(columns = ['Date', 'Towing_and_Articulation', 'Was_Vehicle_Left_Hand_Drive?', 'Driver_IMD_Decile', 'Vehicle_IMD_Decile'], inplace = True, errors = 'ignore')
df_uk_accidents.drop(columns = ['Local_Authority_(District)', 'Local_Authority_(Highway)', 'LSOA_of_Accident_Location'], inplace = True, errors = 'ignore')
df_uk_casualties.drop(columns = ['Casualty_Home_Area_Type', 'Casualty_IMD_Decile'], inplace = True, errors = 'ignore')

In [40]:
df_uk_casualties.head()

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type
0,2017010001708,1,1,Passenger,Female,18.0,16-20,Slight,Not a Pedestrian,Not a Pedestrian,Front seat passenger,Not a bus or coach passenger,No / Not applicable,Car occupant
1,2017010001708,2,2,Driver or rider,Male,19.0,16-20,Serious,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 50cc and under rider or passenger
2,2017010001708,2,3,Passenger,Male,18.0,16-20,Fatal,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 50cc and under rider or passenger
3,2017010009342,1,1,Passenger,Female,33.0,26-35,Slight,Not a Pedestrian,Not a Pedestrian,Front seat passenger,Not a bus or coach passenger,No / Not applicable,Car occupant
4,2017010009344,3,1,Driver or rider,Female,31.0,26-35,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Car occupant


In [42]:
df_uk_vehicles.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Vehicle_Make,Vehicle_Model,Vehicle_Year
0,2017010001708,1,Car,Going ahead other,On main carriageway - not in restricted lane,Not at or within 20 metres of junction,None,None,Did not leave carriageway,None,Front,NaN,Male,24.0,21-25,1997.0,Heavy oil,1.0,NaN,NaN,NaN,2016.0
1,2017010001708,2,Motorcycle <=50cc,Going ahead other,On main carriageway - not in restricted lane,Not at or within 20 metres of junction,Skidded,None,Did not leave carriageway,None,Back,NaN,Male,19.0,16-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017010009342,1,Car,Going ahead other,On main carriageway - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Back,NaN,Male,33.0,26-35,1797.0,Hybrid electric,8.0,Urban area,NaN,NaN,2009.0
3,2017010009342,2,Car,Going ahead other,On main carriageway - not in restricted lane,Approaching junction or waiting/parked at junc...,Skidded,None,Did not leave carriageway,None,Front,NaN,Male,40.0,36-45,2204.0,Heavy oil,12.0,Urban area,NaN,NaN,2005.0
4,2017010009344,1,Car,Going ahead other,On main carriageway - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Front,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_uk_accidents.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident
0,2017010001708,532920.0,196330.0,-0.080107,51.650061,Metropolitan Police,Fatal,2,3,05/08/2017,Saturday,03:12,A,105,Single carriageway,30.0,Not at junction or within 20 metres,NaN,NaN,0,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes
1,2017010009342,526790.0,181970.0,-0.173845,51.522425,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,01:30,A,5,Single carriageway,30.0,T or staggered junction,Give way or uncontrolled,NaN,0,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes
2,2017010009344,535200.0,181260.0,-0.052969,51.514096,Metropolitan Police,Slight,3,1,01/01/2017,Sunday,00:30,A,13,Single carriageway,30.0,T or staggered junction,Give way or uncontrolled,C,0,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes
3,2017010009348,534340.0,193560.0,-0.060658,51.624832,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,01:11,A,1010,Roundabout,30.0,Roundabout,Give way or uncontrolled,B,154,None within 50 metres,"Pelican, puffin, toucan or similar non-junctio...",Darkness - lights lit,Raining no high winds,Wet or damp,None,None,Urban,Yes
4,2017010009350,533680.0,187820.0,-0.072372,51.573408,Metropolitan Police,Serious,1,1,01/01/2017,Sunday,01:42,A,107,Dual carriageway,20.0,Crossroads,Auto traffic signal,A,10,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes


Synchronizing the UK dataset

In [0]:
# Taking only those accident indices from UK_acccidents to other dataframes
accident_indices = df_uk_accidents.Accident_Index.unique()

df_uk_vehicles = df_uk_vehicles[df_uk_vehicles.Accident_Index.isin(accident_indices)]
df_uk_casualties = df_uk_casualties[df_uk_casualties.Accident_Index.isin(accident_indices)]

In [48]:
df_uk_vehicles.isnull().sum()

Accident_Index                            0
Vehicle_Reference                         0
Vehicle_Type                          28409
Vehicle_Manoeuvre                      5979
Vehicle_Location-Restricted_Lane       4815
Junction_Location                     13827
Skidding_and_Overturning               5674
Hit_Object_in_Carriageway             14928
Vehicle_Leaving_Carriageway            5287
Hit_Object_off_Carriageway            97349
1st_Point_of_Impact                    5791
Journey_Purpose_of_Driver           2532911
Sex_of_Driver                        212127
Age_of_Driver                        387800
Age_Band_of_Driver                   387800
Engine_Capacity_(CC)                 900632
Propulsion_Code                      881194
Age_of_Vehicle                      1004798
Driver_Home_Area_Type                704961
Vehicle_Make                        1743013
Vehicle_Model                       1931784
Vehicle_Year                        1004798
dtype: int64

In [49]:
df_uk_accidents.isnull().sum()

Accident_Index                                       0
Location_Easting_OSGR                              137
Location_Northing_OSGR                             137
Longitude                                          147
Latitude                                           147
Police_Force                                         0
Accident_Severity                                    0
Number_of_Vehicles                                   0
Number_of_Casualties                                 0
Date                                                 0
Day_of_Week                                          0
Time                                               138
1st_Road_Class                                  561998
1st_Road_Number                                      0
Road_Type                                        13591
Speed_limit                                         37
Junction_Detail                                  57322
Junction_Control                                698241
2nd_Road_C

In [50]:
df_uk_casualties.isnull().sum()

Accident_Index                              0
Vehicle_Reference                           0
Casualty_Reference                          0
Casualty_Class                              0
Sex_of_Casualty                           709
Age_of_Casualty                         50980
Age_Band_of_Casualty                    50980
Casualty_Severity                           0
Pedestrian_Location                     22564
Pedestrian_Movement                     69015
Car_Passenger                            1373
Bus_or_Coach_Passenger                    128
Pedestrian_Road_Maintenance_Worker    1479706
Casualty_Type                               3
dtype: int64

In [51]:
df_uk_vehicles.describe(include='all')

,Accident_Index,Vehicle_Reference,Vehicle_Type,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type,Vehicle_Make,Vehicle_Model,Vehicle_Year
count,3495851,3.495851e+06,3467442,3489872,3491036,3482024,3490177,3480923,3490564,3398502,3490060,962940,3283724,3.108051e+06,3108051,2.595219e+06,2614657,2.491053e+06,2790890,1752838,1564067,2.491053e+06
unique,1907200,NaN,19,18,10,9,6,11,9,11,5,4,2,NaN,11,NaN,12,NaN,3,524,34116,NaN
top,2013460234852,NaN,Car,Going ahead other,On main carriageway - not in restricted lane,Not at or within 20 metres of junction,None,None,Did not leave carriageway,None,Front,Journey as part of work,Male,NaN,26-35,NaN,Petrol,NaN,Urban area,VAUXHALL,CLIO DYNAMIQUE 16V,NaN
freq,67,NaN,2553163,1651170,3413032,1358552,3038745,3348691,3095716,3201453,1714779,579998,2295963,NaN,709230,NaN,1699305,NaN,2191009,204224,6904,NaN
mean,NaN,1.556312e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.892598e+01,NaN,1.907924e+03,NaN,7.368643e+00,NaN,NaN,NaN,2.002934e+03
std,NaN,7.639325e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.592473e+01,NaN,1.696569e+03,NaN,4.622845e+00,NaN,NaN,NaN,5.557676e+00
min,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN,1.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,1.900000e+03
25%,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.600000e+01,NaN,1.299000e+03,NaN,4.000000e+00,NaN,NaN,NaN,1.999000e+03
50%,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.700000e+01,NaN,1.598000e+03,NaN,7.000000e+00,NaN,NaN,NaN,2.003000e+03
75%,NaN,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.900000e+01,NaN,1.995000e+03,NaN,1.000000e+01,NaN,NaN,NaN,2.006000e+03


In [52]:
df_uk_accidents.describe(include='all')

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident
count,1907200,1.907063e+06,1.907063e+06,1.907053e+06,1.907053e+06,1907200,1907200,1.907200e+06,1.907200e+06,1907200,1907200,1907062,1345202,1.907200e+06,1893609,1.907163e+06,1849878,1208959,361636,1.907200e+06,1904420,1903767,1907186,1824874,1902339,1904471,1904549,1907040,1906930
unique,1907200,NaN,NaN,NaN,NaN,51,3,NaN,NaN,4383,7,1439,5,NaN,5,NaN,8,5,5,NaN,3,6,5,7,5,8,6,2,3
top,2017420242545,NaN,NaN,NaN,NaN,Metropolitan Police,Slight,NaN,NaN,21/10/2005,Friday,17:00,A,NaN,Single carriageway,NaN,Not at junction or within 20 metres,Give way or uncontrolled,A,NaN,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,Yes
freq,1,NaN,NaN,NaN,NaN,291348,1616146,NaN,NaN,822,312809,18613,862449,NaN,1422816,NaN,771296,919658,187516,NaN,1891962,1582119,1393513,1526146,1316860,1858314,1870058,1230259,1525588
mean,NaN,4.410447e+05,2.973981e+05,-1.416007e+00,5.256441e+01,NaN,NaN,1.832976e+00,1.347052e+00,NaN,NaN,NaN,NaN,9.928264e+02,NaN,3.889265e+01,NaN,NaN,NaN,3.700946e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,9.552801e+04,1.607143e+05,1.404082e+00,1.447183e+00,NaN,NaN,7.154178e-01,8.195804e-01,NaN,NaN,NaN,NaN,1.810570e+03,NaN,1.416444e+01,NaN,NaN,NaN,1.284120e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,6.495000e+04,1.029000e+04,-7.516225e+00,4.991294e+01,NaN,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,-1.000000e+00,NaN,1.000000e+01,NaN,NaN,NaN,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,3.775200e+05,1.778500e+05,-2.338432e+00,5.148620e+01,NaN,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,3.000000e+01,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,4.425100e+05,2.624670e+05,-1.370097e+00,5.224767e+01,NaN,NaN,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,1.180000e+02,NaN,3.000000e+01,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,5.240460e+05,3.958600e+05,-2.080870e-01,5.345795e+01,NaN,NaN,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,7.020000e+02,NaN,5.000000e+01,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_uk_casualties.describe(include='all')

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type
count,2569097,2.569097e+06,2.569097e+06,2569097,2568388,2.518117e+06,2518117,2569097,2546533,2500082,2567724,2568969,1089391,2569094
unique,1907200,NaN,NaN,3,2,NaN,11,3,10,9,3,5,2,21
top,20144100J0489,NaN,NaN,Driver or rider,Male,NaN,26-35,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Car occupant
freq,93,NaN,NaN,1625236,1501197,NaN,488722,2244098,2245877,2245877,2039769,2502364,1089062,1571760
mean,NaN,1.485235e+00,1.429836e+00,NaN,NaN,3.535750e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,6.629141e-01,1.368491e+00,NaN,NaN,1.842050e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000e+00,1.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.000000e+00,1.000000e+00,NaN,NaN,2.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.000000e+00,1.000000e+00,NaN,NaN,3.200000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2.000000e+00,2.000000e+00,NaN,NaN,4.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
